# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [2]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [136]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test//2:], newsgroups_test.target[num_test//2:]
dev_data, dev_labels = newsgroups_test.data[:num_test//2], newsgroups_test.target[:num_test//2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print ('training label shape:', train_labels.shape)
print ('test label shape:', test_labels.shape)
print ('dev label shape:', dev_labels.shape)
print ('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [12]:
#def P1(num_examples=5):
### STUDENT START ###

def P1(num_examples = 5):
    
    labels = newsgroups_train['target_names']
    # running through f
    for i in range(num_examples):
        print ('----------------------------------------------------')
        print ('Sample', i, 'has the label', labels[train_labels[i]])
        print ('--- \n')
        print (train_data[i], '\n')
     
P1(5)
        
### STUDENT END ###
#P1(2)

----------------------------------------------------
Sample 0 has the label comp.graphics
--- 

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych 

----------------------------------------------------
Sample 1 has the label talk.religion.misc
--- 



Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [33]:
#def P2():
### STUDENT START ###

def P2():
    vectorizer = CountVectorizer()
    vocab = vectorizer.fit_transform(train_data)

    print ('----- part a -----\n')
    # part a:
    ## What is the size of the vocabulary? 
    print("Shape of vocabulary is: ", vocab.shape, "\n")

    ## What is the average number of non-zero features per example?
    avg_non_zero = np.sum([i.nnz for i in vocab]) / vocab.shape[0]
    print ('Avg no of non-zero features:', avg_non_zero)
    
    ## What fraction of the entries in the matrix are non-zero?
    fraction_non_zero = vocab.nnz / vocab.toarray().size
    print ('Avg no of non-zero features:', fraction_non_zero)
    
    print ('\n----- part b -----\n')   
    # part b:
    ## What are the 0th and last feature strings (in alphabetical order)?
    ## Hint: use the vectorizer's get_feature_names function.
    
    features = vectorizer.get_feature_names()
    print("the 0th feature is:", features[0])
    print("the last feature is:", features[-1])
    
    print ('\n----- part c -----\n')   
    # part c:
    ## Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"].
    print('using 4 word vocabulary:')
    
    vectorizer2 = CountVectorizer(vocabulary = ['atheism', 'graphics', 'space', 'religion'])
    vocab2 = vectorizer2.fit_transform(train_data)
    
    ## Confirm the training vectors are appropriately shaped.
    print("Shape of new vocabulary is:", vocab2.shape)
    
    ## Now what's the average number of non-zero features per example?
    avg_non_zero2 = np.sum([i.nnz for i in vocab2]) / vocab2.shape[0]
    print ('Avg no of non-zero features for this vocabulary:', avg_non_zero2)
    
    print ('\n----- part d -----\n') 
    # part d:
    ## Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features.
    print('using analyzer and ngram_range:')
    
    vectorizer3 = CountVectorizer(analyzer='char', ngram_range=(2, 3))
    vocab3 = vectorizer3.fit_transform(train_data)
    
    ## What size vocabulary does this yield?
    print("Shape of new vocabulary is:", vocab3.shape)
    
    print ('\n----- part e -----\n') 
    # part e:
    ## Use the "min_df" argument to prune words that appear in fewer than 10 documents.
    print('using min_df:')
    
    vectorizer4 = CountVectorizer(min_df = 10 / len(train_data))
    vocab4 = vectorizer4.fit_transform(train_data)
    
    ## What size vocabulary does this yield?
    print("Shape of new vocabulary is:", vocab4.shape)
    
    
    print ('\n----- part f -----\n') 
    # part f:
    ## Using the standard CountVectorizer
    print('using standard CountVectorizer:')
    
    ## Hint: build a vocabulary for both train and dev and look at the size of the difference.
    vocab_f1 = vectorizer.fit_transform(train_data)
    vocab_train = vectorizer.get_feature_names()

    vocab_f2 = vectorizer.fit_transform(dev_data)
    vocab_dev = vectorizer.get_feature_names()
    
    ## what fraction of the words in the dev data are missing from the vocabulary?
    frac_missing = len(set(vocab_dev).difference(set(vocab_train))) / len(vocab_train)
    print ('Fraction of words that are missing from vocab:', frac_missing)
    
P2()
    
### STUDENT END ###
#P2()

----- part a -----

Shape of vocabulary is:  (2034, 26879) 

Avg no of non-zero features: 96.70599803343165
Avg no of non-zero features: 0.0035978272269590263

----- part b -----

the 0th feature is: 00
the last feature is: zyxel

----- part c -----

using 4 word vocabulary:
Shape of new vocabulary is: (2034, 4)
Avg no of non-zero features for this vocabulary: 0.26843657817109146

----- part d -----

using analyzer and ngram_range:
Shape of new vocabulary is: (2034, 35478)

----- part e -----

using min_df:
Shape of new vocabulary is: (2034, 3064)

----- part f -----

using standard CountVectorizer:
Fraction of words that are missing from vocab: 0.14981956173964806


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [81]:
#def P3():
### STUDENT START ###

def P3():
    
    # KNN
    print ('\n----- k nearest neighbor -----\n') 
    
    k_values = [1, 3, 5, 7, 9, 11, 13, 15]
    accuracy = []
    
    vectorizer = CountVectorizer()
    train_vocab = vectorizer.fit_transform(train_data)
    dev_vocab = vectorizer.fit_transform(dev_data)

    for value in k_values:
        k = KNeighborsClassifier(n_neighbors = value)
#         k.fit(train_vocab, train_labels)
        
    # MNB
    print ('\n----- multinomial naive bayes -----\n') 
    
    alphas = {'alpha': [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]}

    MNB = MultinomialNB()
    grid_search = GridSearchCV(MNB.fit(train_vocab, train_labels), param_grid = alphas, refit=True)
    grid_search.fit(dev_vocab, dev_labels)
    
    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        
    print ('\n For a multinomial Naive Bayes, the optimal alpha value is around 0.01')

    # LR
    print ('\n----- logistic regression -----\n') 
    
#     C = {'C': [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0] }
    C = [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]

    logReg = LogisticRegression()
    grid_search_logReg = GridSearchCV(logReg.fit(train_vocab, train_labels), param_grid = C, refit=True)
    grid_search_logReg.fit(dev_vocab, dev_labels)
    
    means_logReg = grid_search_logReg.cv_results_['mean_test_score']
    stds_logReg = grid_search_logReg.cv_results_['std_test_score']
    for mean, std, params in zip(means_logReg, stds_logReg, grid_search_logReg.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        
    # (c)
    print ('\n----- question c -----\n') 
    
    # Output the sum of the squared weight values for each class
    # for each setting of the C parameter.
    
    for i in range(len(C)):
        logReg2 = LogisticRegression(C = C[i])
        logReg2.fit (train_vocab, train_labels)
        print('For C =', C[c], 'the sum of squared weight values is:')
        print('  ', np.sum(np.square(logReg2.coef_),axis = 1))
    
P3()

### STUDENT END ###
#P3()


----- k nearest neighbor -----


----- multinomial naive bayes -----



C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this 

0.762 (+/-0.051) for {'alpha': 0.0001}
0.759 (+/-0.021) for {'alpha': 0.005}
0.765 (+/-0.043) for {'alpha': 0.001}
0.759 (+/-0.021) for {'alpha': 0.005}
0.763 (+/-0.021) for {'alpha': 0.01}
0.754 (+/-0.025) for {'alpha': 0.05}
0.747 (+/-0.040) for {'alpha': 0.1}
0.731 (+/-0.030) for {'alpha': 0.5}
0.728 (+/-0.007) for {'alpha': 1.0}
0.710 (+/-0.048) for {'alpha': 1.5}
0.695 (+/-0.075) for {'alpha': 2.0}
0.592 (+/-0.198) for {'alpha': 5.0}
0.485 (+/-0.236) for {'alpha': 10.0}

 For a multinomial Naive Bayes, the optimal alpha value is around 0.01

----- logistic regression -----



AttributeError: 'float' object has no attribute 'items'

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [70]:
#def P4():
### STUDENT START ###

def P4(my_ngram = (1,1), split = 'word'):
    
    vectorizer = CountVectorizer(ngram_range = my_ngram , analyzer = split)
    train_vocab = vectorizer.fit_transform(train_data)

    logReg = LogisticRegression()
    logReg.fit(train_vocab, train_labels)

    rows = []

    for i in range(len(newsgroups_train.target_names)):
        rows.append(np.argsort(logReg.coef_[i])[-5:])

    rows = [item for sublist in rows for item in sublist]

    table = []

    for i in range(len(newsgroups_train.target_names)):
        for j in range(len(rows)):
            table.append(logReg.coef_[i][rows[j]])

    matrix = np.reshape(table, (-1,4), 'F')

    words = []

    for k in range(len(rows)):
        words.append(vectorizer.get_feature_names()[rows[k]])

    df = pd.DataFrame(matrix, columns=newsgroups_train.target_names)
    df.insert(loc=0, column='Feature', value=words)
    return(df)
  
print('features and weights for each label:')
P4()
    
### STUDENT END ###
#P4()

features and weights for each label:


C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


,Feature,alt.atheism,comp.graphics,sci.space,talk.religion.misc
0,atheism,0.939421,-0.410411,-0.449127,-0.434528
1,religion,0.953546,-0.616899,-0.792985,-0.064419
2,bobby,0.989721,-0.220738,-0.341069,-0.463548
3,atheists,1.031080,-0.097768,-0.318993,-0.834724
4,deletion,1.124811,-0.398148,-0.420192,-0.395484
5,computer,0.143923,0.977781,-0.682015,-0.487105
6,3d,-0.358972,1.125111,-0.702499,-0.378458
7,file,-0.334730,1.266608,-0.806897,-0.626653
8,image,-0.582941,1.346867,-0.825001,-0.468532
9,graphics,-0.758369,1.936757,-1.336521,-0.762745


In [71]:
print('bigram features and weights for each label:')
P4(my_ngram = (1,2), split = 'word')

bigram features and weights for each label:


C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


,Feature,alt.atheism,comp.graphics,sci.space,talk.religion.misc
0,up,0.598964,-0.202332,-0.304278,-0.240782
1,atheists,0.610580,-0.094831,-0.200779,-0.393019
2,atheism,0.645665,-0.248437,-0.280090,-0.284580
3,religion,0.755124,-0.420451,-0.518832,-0.027082
4,deletion,1.179178,-0.395394,-0.389776,-0.379618
5,hi,-0.256537,0.690073,-0.421185,-0.319588
6,3d,-0.241557,0.765136,-0.476942,-0.248029
7,image,-0.391771,0.942855,-0.536170,-0.358559
8,file,-0.265345,0.944724,-0.613214,-0.461888
9,graphics,-0.568687,1.511615,-0.995050,-0.560648


ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [111]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
### STUDENT START ###
    
    # try lowercasing everything
    s = s.lower()
    
    # replacing sequences of numbers with a single token
    s = re.sub(r"\b\d+\b", "numeric_string", s)
    
    return s

### STUDENT END ###

### STUDENT START ###
def P5():
    vectorizer = CountVectorizer(preprocessor = empty_preprocessor)
    train_vocab = vectorizer.fit_transform(train_data)
    vocab = vectorizer.vocabulary_
    
    dev_vocab = vectorizer.transform(dev_data)
    
    logReg = LogisticRegression(C = 0.15)
    logReg.fit(train_vocab, train_labels)

    f1 = round(metrics.f1_score(dev_labels, logReg.predict(dev_vocab), average='weighted')*100, 3)
    
    print('f1 score with empty preprocessor:', f1, '%')
    
    # repeat with better preprocessor
    
    vectorizer2 = CountVectorizer(preprocessor = better_preprocessor)
    train_vocab2 = vectorizer2.fit_transform(train_data)
    vocab2 = vectorizer2.vocabulary_
    
    dev_vocab2 = vectorizer2.transform(dev_data)
    
    logReg2 = LogisticRegression(C = 0.15)
    logReg2.fit(train_vocab2, train_labels)
    
    f1_2 = round(metrics.f1_score(dev_labels, logReg2.predict(dev_vocab2), average='weighted')*100, 3)
    
    print('f1 score with better preprocessor:', f1_2, '%')
    
    difference = f1_2 - f1
    print('improvement size:', round(difference, 3), '%')
    
### STUDENT END ###
P5()

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


f1 score with empty preprocessor: 70.408 %
f1 score with better preprocessor: 70.706 %
improvement size: 0.298 %


(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [145]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###

    # Train a logistic regression model using a "l1" penalty.
    C = [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]
        
    sizes = []
    scores = []
    
    for c in C:
        vectorizer = CountVectorizer()
        train_vocab = vectorizer.fit_transform(train_data)
        vocab = vectorizer.vocabulary_
        
        logReg = LogisticRegression(penalty = 'l1', C = c, tol = 0.01)
        logReg.fit(train_vocab, train_labels)    
    
        i,j = np.nonzero(logReg.coef_)
        new_vocab = {k:v for k,v in vocab.items() if v in j}
        sizes.append(len(new_vocab.keys()))
        
        # train l2 model with new vocab
        vectorizer2 = CountVectorizer(vocabulary = new_vocab.keys())
        
        # the following command throws up the error: empty vocabulary passed to fit
        # why? train_data is clearly not empty
        # COME BACK TO THIS
        train_vocab2 = vectorizer2.fit_transform(train_data)
        
        dev_vocab2 = vectorizer2.transform(dev_data)
    
#     # plot it    
#     plt.plot(sizes, scores)
        
    ### STUDENT END ###
P6()

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


ValueError: empty vocabulary passed to fit

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [164]:
def P7():
    ### STUDENT START ###

    tfid = TfidfVectorizer()
    train_vocab = tfid.fit_transform(train_data)
    dev_vocab = tfid.transform(dev_data)
    
    # Train a logistic regression model with C=100.
    logReg = LogisticRegression(C = 100)
    logReg.fit(train_vocab, train_labels)
    
    # Make predictions on the dev data
    prediction = logReg.predict(dev_vocab)
    
    ## maximum predicted probability 
    probabilities = logReg.predict_proba(dev_vocab)
    probs_max = np.max(probabilities, axis = 1)
    
    ## predicted probability of the correct label
    probs_correct = np.array([probabilities[i, dev_labels[i]] for i in range(len(dev_labels))])

    # show the top 3 documents where the ratio R is largest
    # where R is:
    R = probs_max / probs_correct
    top3 = np.argsort(R)[::-1][:3]
    names = newsgroups_train['target_names']
    for i in range(3):
        print('In document', i, ':')
        print('  actual label:', names[dev_labels[top3[i]]])
        print('  predicted label:', names[prediction[top3[i]]])
        print('  R:', R[top3[i]])
        print('\n', dev_data[top3[i]], '\n ---------------------------')
    
    ### STUDENT END ###
P7()

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In document 0 :
  actual label: talk.religion.misc
  predicted label: comp.graphics
  R: 929.3580170576168

 I am pleased to announce that a *revised version* of _The Easy-to-Read Book
of Mormon_ (former title: _Mormon's Book_) by Lynn Matthews Anderson is now
available through anonymous ftp (see information below). In addition to the
change in title, the revised ETR BOM has been shortened by several pages
(eliminating many extraneous "that's" and "of's"), and many (minor) errors
have been corrected. This release includes a simplified Joseph Smith Story,
testimonies of the three and eight witnesses, and a "Words-to-Know"
glossary.

As with the previous announcement, readers are reminded that this is a
not-for-profit endeavor. This is a copyrighted work, but people are welcome
to make *verbatim* copies for personal use. People can recuperate the
actual costs of printing (paper, copy center charges), but may not charge
anything for their time in making copies, or in any way realize a pro

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance